In [1]:
%run "Setup_Env.ipynb"

## Branching and Merging Chains with LCEL

The idea here is to have multiple branching LLM Chains which work independently in parallel and then we merge their outputs finally using a merge LLM chain at the end to get a consolidated output

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
description_prompt =  ChatPromptTemplate.from_template(
    """Generate a two line description for the given topic:
      {topic}
""")

description_chain = (
    description_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

In [5]:
pro_prompt = ChatPromptTemplate.from_template(
    """Generate three bullet points talking about the pros for the given topic:
      {topic}
""")

pro_chain = (
    pro_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

In [6]:
con_prompt = ChatPromptTemplate.from_template(
    """Generate three bullet points talking about the cons for the given topic:
      {topic}
""")

con_chain = (
    con_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

In [7]:
from operator import itemgetter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

branch_chain = (
    RunnableParallel(
        topic=itemgetter('topic'),
        description=description_chain,
        pros=pro_chain,
        cons=con_chain,
    )
)

In [8]:
branch_chain = (
    RunnableParallel(
        topic=RunnablePassthrough(),
        description=description_chain,
        pros=pro_chain,
        cons=con_chain,
    )
)

In [9]:
branch_chain.invoke({"topic": "Generative AI"})

{'topic': {'topic': 'Generative AI'},
 'description': 'Generative AI refers to advanced algorithms that create new content, such as text, images, and music, by learning patterns from existing data. This technology has transformative applications across various fields, including art, entertainment, and business.',
 'pros': '- **Enhanced Creativity and Innovation**: Generative AI can assist artists, writers, and designers by providing new ideas, concepts, and variations, thereby fostering creativity and enabling the exploration of uncharted territories in various fields.\n\n- **Efficiency and Automation**: By automating repetitive tasks and generating content at scale, generative AI can significantly reduce the time and effort required for content creation, allowing professionals to focus on higher-level strategic work and decision-making.\n\n- **Personalization and User Engagement**: Generative AI can analyze user data to create highly personalized experiences, such as tailored recommen

In [10]:
merge_prompt = ChatPromptTemplate.from_template(
    """Create a report about {topic} with the following information:
      Description:
      {description}
      Pros:
      {pros}
      Cons:
      {cons}

      Report should be in the following format:

      Topic: <name of the topic>

      Description: <description of the topic>

      Pros and Cons:

      <table with two columns showing the 3 pros and cons of the topic>
""")

merge_chain = (
    merge_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

In [11]:
final_chain = (
    branch_chain
      |
    merge_chain
)

In [12]:
response = final_chain.invoke({"topic": "Generative AI"})

In [13]:
from IPython.display import Markdown, display

display(Markdown(response))

# Report on Generative AI

**Topic:** Generative AI

**Description:**  
Generative AI refers to advanced algorithms that create new content, such as text, images, and music, by learning patterns from existing data. This technology has transformative applications across various fields, including art, entertainment, and business.

**Pros and Cons:**

| **Pros**                                           | **Cons**                                           |
|---------------------------------------------------|---------------------------------------------------|
| **Enhanced Creativity and Innovation**: Generative AI can produce unique content, designs, and solutions that may not be easily conceived by humans, fostering creativity and driving innovation across various fields such as art, music, and product design. | **Ethical Concerns**: Generative AI can produce content that may perpetuate biases, misinformation, or harmful stereotypes, raising ethical questions about accountability and the potential for misuse in various applications, such as deepfakes or propaganda. |
| **Increased Efficiency and Productivity**: By automating repetitive tasks and generating high-quality outputs quickly, generative AI can significantly reduce the time and effort required for content creation, allowing professionals to focus on more strategic and complex aspects of their work. | **Intellectual Property Issues**: The ability of generative AI to create content that closely resembles existing works can lead to legal disputes over copyright and ownership, complicating the landscape for creators and businesses relying on original content. |
| **Personalization and Customization**: Generative AI enables the creation of highly personalized experiences and products tailored to individual preferences, enhancing customer satisfaction and engagement in industries like marketing, entertainment, and e-commerce. | **Job Displacement**: As generative AI becomes more capable of performing tasks traditionally done by humans, there is a growing concern about job displacement in creative fields, potentially leading to economic instability and a shift in workforce demands. |